## Import data 

In [1]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_targets_FILENAME = r'data\target_list_filtered.xlsx'
df_exoclock_FILENAME = r'data\exoclock_all_mid_time_data.txt'
df_holczer_FILENAME = r'data\Holczer-TTV_data.tsv'
df_berkaoui_FILENAME = r'data\TOI2015b-ephemerides-berkaoui25.csv'
df_KOImap_FILENAME = r'data\KOI-Kepler-map_2026.01.19_05.52.41.csv'

In [3]:
# df_targets = pd.read_csv(df_targets_FILENAME)
df_targets = pd.read_excel(df_targets_FILENAME)
df_targets.tail(2)

,KOI_inner,KOI_outer,name_inner,name_outer,T0_inner,T0_inner_err,T0_outer,T0_outer_err,P_inner,P_inner_err,...,peri_inner_err,peri_outer,peri_outer_err,mutual_inclination,mutual_inclination_err,st_mass,st_mass_error,parameter_reference,parameter_reference_doi,Notes
12,NaN,NaN,TOI-2015b,TOI-2015c,2.459713e+06,0.00251,NaN,NaN,3.346493,0.000047,...,1.260997,-83.602646,1.886108,NaN,NaN,0.33,0.02,Berkaoui+25,10.1051/0004-6361/202452916,"ew_converted, High mutual inclination"
13,NaN,NaN,TOI-2015b,TOI-2015c,2.459713e+06,0.00251,NaN,NaN,3.348004,0.000056,...,1.346652,-26.748905,2.498920,NaN,NaN,0.33,0.02,Berkaoui+25,10.1051/0004-6361/202452916,"ew_converted, High mutual inclination"


In [4]:
df_exoclock = pd.read_csv(df_exoclock_FILENAME, sep='\s+')

df_exoclock.tail(3)

,Planet,Tmid_(BJD_TDB),Tmid_unc.,source,ID
30341,piMenc,2.460149e+06,0.00084,space,piMenc_1372_2023-07-23_TESS
30342,piMenc,2.460156e+06,0.00095,space,piMenc_1373_2023-07-30_TESS
30343,piMenc,2.460175e+06,0.00110,space,piMenc_1376_2023-08-18_TESS


In [5]:
holczer_cols = ['KOI', 'tn', 'N', 'O-C', 'e_O-C']
df_holczer = pd.read_csv(df_holczer_FILENAME, sep=';', comment='#', header=0, low_memory=False,
                     )[2:][holczer_cols].reset_index(drop=True)
# Type set
df_holczer['KOI'] = df_holczer['KOI'].astype(str).str.strip()
df_holczer['N'] = df_holczer['N'].astype(int)
df_holczer['tn'] = df_holczer['tn'].astype(float) + 2454900
min2day = 1/1440
df_holczer[['O-C', 'e_O-C']] = df_holczer[['O-C', 'e_O-C']].astype(float) * min2day

df_holczer.head(3)

,KOI,tn,N,O-C,e_O-C
0,1.01,2.454956e+06,0,-0.000035,0.000063
1,1.01,2.454958e+06,1,0.000053,0.000063
2,1.01,2.454961e+06,2,-0.000026,0.000063


In [6]:
df_berkaoui = pd.read_csv(df_berkaoui_FILENAME); df_berkaoui.tail(2)

,epoch,Tmid_(BJD_TDB),Tmid_unc.,Telescope
48,218,2.460443e+06,0.0023,TESS
49,220,2.460449e+06,0.0015,TESS


In [7]:
map_cols = ['kepoi_name', 'kepler_name']
df_KOImap = pd.read_csv(df_KOImap_FILENAME, comment='#')[map_cols]
# Extract KOI number from kepoi_name
df_KOImap['KOI_no'] = (
    df_KOImap['kepoi_name']
    .astype(str)
    .str.extract(r'K*(\d+\.\d+)')[0]
    .astype(float).astype(str)
)
df_KOImap['kepler_name'] = (
    df_KOImap['kepler_name']
    .astype(str)
    .str.replace(r'\s+', '', regex=True)
)
koi2kepler_map = dict(
    df_KOImap[["KOI_no", "kepler_name"]]
    .itertuples(index=False, name=None)
)
koi2kepler_map

{'752.01': 'Kepler-227b',
 '752.02': 'Kepler-227c',
 '755.01': 'Kepler-664b',
 '756.01': 'Kepler-228d',
 '756.02': 'Kepler-228c',
 '756.03': 'Kepler-228b',
 '757.01': 'Kepler-229c',
 '1.01': 'Kepler-1b',
 '2.01': 'Kepler-2b',
 '10.01': 'Kepler-8b',
 '112.02': 'Kepler-466c',
 '746.01': 'Kepler-660b',
 '747.01': 'Kepler-661b',
 '749.01': 'Kepler-226c',
 '749.02': 'Kepler-226b',
 '749.03': 'Kepler-226d',
 '750.01': 'Kepler-662b',
 '751.01': 'Kepler-663b',
 '769.01': 'Kepler-671b',
 '773.01': 'Kepler-672b',
 '757.02': 'Kepler-229d',
 '757.03': 'Kepler-229b',
 '758.01': 'Kepler-665b',
 '759.01': 'Kepler-230b',
 '759.02': 'Kepler-230c',
 '760.01': 'Kepler-1976b',
 '762.01': 'Kepler-666b',
 '115.01': 'Kepler-105b',
 '764.01': 'Kepler-667b',
 '765.01': 'Kepler-668b',
 '766.01': 'Kepler-669b',
 '767.01': 'Kepler-670b',
 '786.01': 'Kepler-680b',
 '787.01': 'Kepler-232b',
 '787.02': 'Kepler-232c',
 '788.01': 'Kepler-681b',
 '116.01': 'Kepler-106c',
 '115.02': 'Kepler-105c',
 '775.01': 'Kepler-52c

## Merge 3 different sources into single, clean format

In [8]:
# Unique targets and corresponding periods
exceptions = {"TOI-2015c", "WASP-148c"}
targets_periods_errs = list({
    (row["name_inner"], row["T0_inner"], row["T0_inner_err"], row["P_inner"], row["P_inner_err"])
    for _, row in df_targets[["name_inner", "T0_inner", "T0_inner_err", "P_inner", "P_inner_err"]].dropna().iterrows()
    if row["name_inner"] not in exceptions
}.union({
    (row["name_outer"], row["T0_outer"], row["T0_outer_err"], row["P_outer"], row["P_outer_err"])
    for _, row in df_targets[["name_outer", "T0_outer", "T0_outer_err" , "P_outer", "P_outer_err"]].dropna().iterrows()
    if row["name_outer"] not in exceptions
}))

targets, T0, T0_errs, periods, period_errs = zip(*targets_periods_errs)

T0_map = dict(zip(targets, T0))
T0_err_map = dict(zip(targets, T0_errs))
period_map = dict(zip(targets, periods))
period_err_map = dict(zip(targets, period_errs))

print(targets); print(periods); print(period_errs)

('TOI-2015b', 'KOI-94c', 'Kepler-36b', 'Kepler-49c', 'Kepler-238e', 'Kepler-49b', 'Kepler-36c', 'Kepler-307c', 'Kepler-18c', 'Kepler-26c', 'TOI-2015b', 'Kepler-18d', 'Kepler-307b', 'WASP-148b', 'Kepler-238f', 'Kepler-52b', 'Kepler-54b', 'TOI-2015b', 'KOI-94d', 'Kepler-54c', 'Kepler-52c', 'Kepler-138d', 'Kepler-26b', 'Kepler-138c')
(3.34649323, 10.423674, 13.84911836, 10.91271752, 23.65367567, 7.20385258, 16.23189245, 13.08428012, 7.64156219, 17.25118601, 3.348237, 14.8589225, 10.41573989, 8.803812, 50.44727195, 7.87742645, 8.01080765, 3.34800359, 22.3429719, 12.07130385, 16.38482024, 23.08898875, 12.2830153, 13.7810915)
(4.738e-05, 1.3e-05, 3.056e-05, 3.39e-06, 1.06e-06, 1.73e-06, 1.597e-05, 2.54e-06, 5.7e-07, 2.11e-06, 3.5e-05, 9.2e-06, 1.43e-06, 1e-05, 1.573e-05, 2.48e-06, 6.63e-06, 5.563e-05, 5.2e-06, 1.547e-05, 3.9e-06, 3.16e-06, 1.67e-06, 8.9e-07)


In [9]:
# Prepare exoclock
df_exoclock_2 = (
    df_exoclock[df_exoclock["Planet"].isin(targets)]
    .assign(
        T0=lambda d: d["Planet"].map(T0_map),
        T0_err=lambda d: d["Planet"].map(T0_err_map),
        period=lambda d: d["Planet"].map(period_map),
        period_err=lambda d: d["Planet"].map(period_err_map),
    )
    .reset_index(drop=True)
)
# Acquire epoch and finally O-C [+err]
df_exoclock_2['epoch'] = np.int64(np.round(
    (df_exoclock_2['Tmid_(BJD_TDB)'] - df_exoclock_2['T0']) / df_exoclock_2['period']
))
df_exoclock_2['O-C_(days)'] = df_exoclock_2['Tmid_(BJD_TDB)'] -\
    (df_exoclock_2['T0'] + df_exoclock_2['epoch']*df_exoclock_2['period']) 	
df_exoclock_2['O-C_err_(days)'] = df_exoclock_2['Tmid_unc.'] + df_exoclock_2['T0_err'] + np.abs(
    (df_exoclock_2['epoch'] * df_exoclock_2['period_err'])
)

# Drop unused columns
df_exoclock_2 = df_exoclock_2.drop(columns=['ID', 'T0', 'T0_err'])

# Rename source
df_exoclock_2['source'] = (
    df_exoclock_2['source']
    .str.replace(r'\bliterature\b', 'Kokori+25 (literature)', regex=True)
    .str.replace(r'\bexoclock\b', 'Kokori+25 (exoclock)', regex=True)
    .str.replace(r'\bspace\b', 'Kokori+25 (space)', regex=True)
)

# Rename cols
df_exoclock_2 = df_exoclock_2.rename(
        columns={
            "Tmid_(BJD_TDB)": "Tmid",
            'Tmid_unc.': 'Tmid_err'})

# Rearrange columns
df_exoclock_2 = df_exoclock_2[["Planet", "source", "Tmid", "Tmid_err", "period", "period_err", "epoch", "O-C_(days)", "O-C_err_(days)"]]

df_exoclock_2.head()

,Planet,source,Tmid,Tmid_err,period,period_err,epoch,O-C_(days),O-C_err_(days)
0,KOI-94c,Kokori+25 (space),2.454971e+06,0.0020,10.423674,0.000013,-68,-0.004168,0.003384
1,KOI-94c,Kokori+25 (space),2.454981e+06,0.0018,10.423674,0.000013,-67,-0.001842,0.003171
2,KOI-94c,Kokori+25 (space),2.454992e+06,0.0022,10.423674,0.000013,-66,-0.000116,0.003558
3,KOI-94c,Kokori+25 (space),2.455190e+06,0.0016,10.423674,0.000013,-47,0.000478,0.002711
4,KOI-94c,Kokori+25 (space),2.455200e+06,0.0035,10.423674,0.000013,-46,0.004804,0.004598


In [10]:
# Prepare holczer
df_holczer_2 = df_holczer.copy()
df_holczer_2["Planet"] = df_holczer_2["KOI"].map(koi2kepler_map)  # Map

# Insert period and period_err
df_holczer_2 = df_holczer_2[df_holczer_2["Planet"].isin(targets)] \
    .assign(
        period=lambda d: d["Planet"].map(period_map),
        period_err=lambda d: d["Planet"].map(period_err_map),
    ).reset_index(drop=True)

# Source and Tmid_err
df_holczer_2['source'] = 'Holczer+16'
df_holczer_2['Tmid_err'] = np.nan

# Match column information
df_holczer_2 = (
    df_holczer_2.rename(
        columns={
            'tn': 'Tmid',
            "N": "epoch",
            "O-C": "O-C_(days)",
            "e_O-C": "O-C_err_(days)",
        }
    )
    .loc[:, ["Planet", "source", "Tmid", "Tmid_err", "period", "period_err", "epoch", "O-C_(days)", "O-C_err_(days)"]]
    .reset_index(drop=True)
)

df_holczer_2.head()

,Planet,source,Tmid,Tmid_err,period,period_err,epoch,O-C_(days),O-C_err_(days)
0,Kepler-18c,Holczer+16,2.454961e+06,NaN,7.641562,5.700000e-07,0,0.001181,0.001389
1,Kepler-18c,Holczer+16,2.454968e+06,NaN,7.641562,5.700000e-07,1,-0.001250,0.000694
2,Kepler-18c,Holczer+16,2.454976e+06,NaN,7.641562,5.700000e-07,2,-0.024722,0.000694
3,Kepler-18c,Holczer+16,2.454984e+06,NaN,7.641562,5.700000e-07,3,0.000347,0.001389
4,Kepler-18c,Holczer+16,2.454991e+06,NaN,7.641562,5.700000e-07,4,0.001319,0.001389


In [11]:
# Prepare Berkaoui
df_berkaoui_2 = df_berkaoui.copy()
df_berkaoui_2['Planet'] = 'TOI-2015b'
df_berkaoui_2['source'] = 'Berkaoui+25'

# Assign T0 and P MANUALLY based on 5:3 resonance (map isnt needed!)
df_berkaoui_2 = df_berkaoui_2.assign(
    T0=lambda d: 2459712.9225975,
    T0_err=lambda d: 0.00251,
    period=lambda d: 3.348237,
    period_err=lambda d: 0.000035
).reset_index(drop=True)

# Acquire epoch and finally O-C [+err]
df_berkaoui_2['O-C_(days)'] = df_berkaoui_2['Tmid_(BJD_TDB)'] -\
    (df_berkaoui_2['T0'] + df_berkaoui_2['epoch']*df_berkaoui_2['period']) 	
df_berkaoui_2['O-C_err_(days)'] = df_berkaoui_2['Tmid_unc.'] + df_berkaoui_2['T0_err'] + np.abs(
    (df_berkaoui_2['epoch'] * df_berkaoui_2['period_err'])
)
# Rename cols
df_berkaoui_2 = df_berkaoui_2.rename(
        columns={
            "Tmid_(BJD_TDB)": "Tmid",
            'Tmid_unc.': 'Tmid_err'})

# Match column information
df_berkaoui_2 = df_berkaoui_2[["Planet", "source", "Tmid", "Tmid_err", "period", "period_err", "epoch", "O-C_(days)", "O-C_err_(days)"]]

df_berkaoui_2.head()

,Planet,source,Tmid,Tmid_err,period,period_err,epoch,O-C_(days),O-C_err_(days)
0,TOI-2015b,Berkaoui+25,2.458956e+06,0.00311,3.348237,0.000035,-226,-0.187326,0.013530
1,TOI-2015b,Berkaoui+25,2.458959e+06,0.00517,3.348237,0.000035,-225,-0.195609,0.015555
2,TOI-2015b,Berkaoui+25,2.458963e+06,0.00549,3.348237,0.000035,-224,-0.189347,0.015840
3,TOI-2015b,Berkaoui+25,2.458966e+06,0.00337,3.348237,0.000035,-223,-0.193363,0.013685
4,TOI-2015b,Berkaoui+25,2.458969e+06,0.00285,3.348237,0.000035,-222,-0.199775,0.013130


In [12]:
# Merge!
df_unified = (
    pd.concat([df_exoclock_2, df_holczer_2, df_berkaoui_2], ignore_index=True)
      .reset_index(drop=True)
)
df_unified.to_csv(r'data/all_mid_time_data_with_OC.csv', index=False)